# Introduction

The Grecian Computer is a math puzzle that challenges where the dials have to be rotated so that all of the columns add up to 42. This notebook contains a computational solution to the Grecian computer puzzle.

In [1]:
from scipy.optimize import minimize 
from sympy import *
import math
import numpy as np
import itertools

# Set up

The Grecian computer has 5 dials with the dials stacked on top of one another. We will refer to the bottom most dial as 'Dial 1', the one stacked on top of 'Dial 1' as 'Dial 2', the one stacked on top of 'Dial 2' as 'Dial 3' and so on. We will treat the dials as a $5 \times 12$ matrix, that is, we can view 'Dial 1' as the following matrix:

$$
\begin{bmatrix}
    8 &3 &4 &12 &2 & 5 & 10 & 7 & 16 &8 &7&8\\
    4&4&6&6&3&3&14&14&21&21&9&9\\
    4&5&6&7&8&9&10&11&12&13&14&15\\
    11&11&14&11&14&11&14&14&11&14&11&14
\end{bmatrix}
$$

The dials other than 'Dial 1' have some rows missing or have holes in certain places. I model the holes and missing rows as zeros. For example, 'Dial 5' will have the following matrix representation: 

$$
\begin{bmatrix}
    0&0&0&0&0&0&0&0&0&0&0&0\\
    0&0&0&0&0&0&0&0&0&0&0&0\\
    0&0&0&0&0&0&0&0&0&0&0&0\\
    7&0&15&0&8&0&3&0&6&0&10&0
\end{bmatrix}
$$

In [2]:
# Initial set up of the dials 

dial1_init = np.array([[8,3,4, 12, 2, 5, 10, 7, 16, 8, 7, 8], 
                   [4,4,6,6,3,3,14,14,21,21,9,9], 
                   [4,5,6,7,8,9,10,11,12,13,14,15], 
                   [11,11,14,11,14,11,14,14,11,14,11,14]])

dial2_init = np.array([[10,0,10,0,1,0,9,0,12,0,6,0], 
                   [17,19,3,12,3,26,6,0,2,13,9,0],
                   [3,8,9,0,9,20,12,3,6,0,14,12], 
                   [8,0,16,2,7,0,9,0,7,14,11,0]])


dial3_init = np.array([[0,0,0,0,0,0,0,0,0,0,0,0], 
                   [10,0,8,0,22,0,16,0,9,0,5,0], 
                   [15,4,9,18,11,26,14,1,12,0,21,6], 
                   [9,7,13,21,17,4,5,0,7,8,9,13]])


dial4_init = np.array([[0,0,0,0,0,0,0,0,0,0,0,0], 
                   [0,0,0,0,0,0,0,0,0,0,0,0], 
                   [14,0,9,0,12,0, 4, 0, 7, 15, 0, 0], 
                   [11,6,11,0,6,17,7,3,0,6,0,11]])



dial5_init = np.array([[0,0,0,0,0,0,0,0,0,0,0,0],
                   [0,0,0,0,0,0,0,0,0,0,0,0],
                   [0,0,0,0,0,0,0,0,0,0,0,0],
                   [7,0,15,0,8,0,3,0,6,0, 10,0]])

# Adding up

Given any set up of the dials, we can calculate the sum of each column the following way: 

1. Go through each row (there are 5 rows)
2. Find the 'highest' dial which has a non-zero value for that row and record that value. 
3. Add up the values from 2. 

When the dials are stacked on top of one another, the value of the top most dial is added to the sum unless there is a hole in that dial. Step 2 captures this design of the puzzle. 

In [3]:
def adding_up(dial_1, dial_2, dial_3, dial_4, dial_5): 
    vals = np.zeros((4,12))
    for i in range(4):
        for j in range(12): 
            dd_1 = dial_1[i,j]
            dd_2 = dial_2[i,j]
            dd_3 = dial_3[i,j]
            dd_4 = dial_4[i,j]
            dd_5 = dial_5[i,j]
            lst = [dd_1, dd_2, dd_3, dd_4, dd_5]
            
            # Find the 'highest' dial which has a non-zero value 
            ind = np.where(lst)[0].max()
            
            # Add that to the values that are going to be added 
            vals[i,j] = lst[ind]
    tot = np.sum(vals, axis=0)
    return tot

# Shifting Right

To solve the puzzles, we have to consider permutations of the dials such that the previous step of adding up yields 42 for all columns. When considering permutations, we can treat 'Dial 1', the bottom most dial, as being fixed. We will rotate the remaining four dials and there are 12 possible rotations for each dial. We define the following function which shifts the values of each row to the right. For example, passing 'Dial 5' once through the shift_right function we will get:

$$
\begin{bmatrix}
    0&0&0&0&0&0&0&0&0&0&0&0\\
    0&0&0&0&0&0&0&0&0&0&0&0\\
    0&0&0&0&0&0&0&0&0&0&0&0\\
    0&7&0&15&0&8&0&3&0&6&0&10
\end{bmatrix}
$$

In [4]:
def shift_right(dial):
    new_dial = np.zeros(dial.shape)
    new_dial[:,0]=dial[:,-1]
    new_dial[:,1:]=dial[:,:-1]
    return new_dial

# Solving the puzzle 

All that is left to do is to go over all the permutations. We consider permutations of the dials by rotating them to the right and calculating the sum of each column for every permutation. There are $12 \times 12 \times 12 \times 12 = 20,736$ possibilies and we continue permutating until we reach the solution: sum of each column equaling 42. 

In [7]:
x2 = dial2_init

x3  = dial3_init

x4 = dial4_init

x5 = dial5_init

for i in range(12):
    x2  = shift_right(x2)
    for j in range(12):
        x3  = shift_right(x3)
        for k in range(12):
            x4 = shift_right(x4)
            for l in range(12):
                x5 = shift_right(x5)
                val = adding_up(dial1_init, x2, x3, x4, x5)
                if np.all(val == 42):
                    print('Done:', i, j, k, l)
                    print('Sum list:', val)
                    print('Dial 1:', dial1_init)
                    print('Dial 2:', x2)
                    print('Dial 3:', x3)
                    print('Dial 4:', x4)
                    break

Done: 11 11 10 11
Sum list: [42. 42. 42. 42. 42. 42. 42. 42. 42. 42. 42. 42.]
Dial 1: [[ 8  3  4 12  2  5 10  7 16  8  7  8]
 [ 4  4  6  6  3  3 14 14 21 21  9  9]
 [ 4  5  6  7  8  9 10 11 12 13 14 15]
 [11 11 14 11 14 11 14 14 11 14 11 14]]
Dial 2: [[10.  0. 10.  0.  1.  0.  9.  0. 12.  0.  6.  0.]
 [17. 19.  3. 12.  3. 26.  6.  0.  2. 13.  9.  0.]
 [ 3.  8.  9.  0.  9. 20. 12.  3.  6.  0. 14. 12.]
 [ 8.  0. 16.  2.  7.  0.  9.  0.  7. 14. 11.  0.]]
Dial 3: [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [10.  0.  8.  0. 22.  0. 16.  0.  9.  0.  5.  0.]
 [15.  4.  9. 18. 11. 26. 14.  1. 12.  0. 21.  6.]
 [ 9.  7. 13. 21. 17.  4.  5.  0.  7.  8.  9. 13.]]
Dial 4: [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  9.  0. 12.  0.  4.  0.  7. 15.  0.  0. 14.]
 [ 6. 11.  0.  6. 17.  7.  3.  0.  6.  0. 11. 11.]]
